In [15]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import csv

In [4]:
# Importing csv with judge names and disclosure availability status
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
grandparent_dir = os.path.dirname(parent_dir)
data_dir = os.path.join(grandparent_dir, 'data/persons_positions/people_positions_disc_summary.csv')
print(data_dir)

/Users/eshan23/eshanprashar_git_profile/judges-conflicts/data/persons_positions/people_positions_disc_summary.csv


In [6]:
# Import the csv
df_jud_fin_status = pd.read_csv(data_dir)
df_jud_fin_status.head()

,person_id,judge_flag,position,status,disclosures_count,name_first,name_middle,name_last,political_affiliations,race
0,1,0,1,non-judge,NaN,George,NaN,Washington,f,w
1,2,0,1,non-judge,NaN,John,NaN,Adams,f,w
2,3,0,1,non-judge,NaN,Thomas,NaN,Jefferson,j,w
3,4,0,1,non-judge,NaN,James,NaN,Madison,j,w
4,5,0,1,non-judge,NaN,James,NaN,Monroe,j,w


In [8]:
# Created a new column called name and extract that in a list
df_jud_fin_status['name'] = df_jud_fin_status['name_first'] + ' ' + df_jud_fin_status['name_last']
judges = df_jud_fin_status['name'].to_list()
print(judges)

['George Washington', 'John Adams', 'Thomas Jefferson', 'James Madison', 'James Monroe', 'John Adams', 'Andrew Jackson', 'Martin Buren', 'William Harrison', 'John Tyler', 'James Polk', 'Zachary Taylor', 'Millard Fillmore', 'Franklin Pierce', 'James Buchanan', 'Abraham Lincoln', 'Andrew Johnson', 'Ulysses Grant', 'Rutherford Hayes', 'James Garfield', 'Chester Arthur', 'Grover Cleveland', 'Benjamin Harrison', 'William McKinley', 'Theodore Roosevelt', 'William Taft', 'Woodrow Wilson', 'Warren Harding', 'Calvin Coolidge', 'Herbert Hoover', 'Franklin Roosevelt', 'Harry Truman', 'Dwight Eisenhower', 'John Kennedy', 'Lyndon Johnson', 'Richard Nixon', 'Gerald Ford', 'Jimmy Carter', 'Ronald Reagan', 'George Bush', 'William Clinton', 'George Bush', 'Barack Obama', 'Leslie Abrams', 'Ronnie Abrams', 'Matthew Abruzzo', 'Marcus Acheson', 'William Acker', 'Harold Ackerman', 'James Ackerman', 'Raymond Acosta', 'Jackson Adair', 'Arlin Adams', 'Elmer Adams', 'George Adams', 'George Adams', 'Henry Adams'

In [10]:
driver = webdriver.Chrome()
url = 'https://pub.jefs.uscourts.gov/#'
driver.get(url)
time.sleep(5)

In [13]:
def search_name(name):
    try:
        # Open the website
        driver.get("https://pub.jefs.uscourts.gov/#")

        # Locate the search input field and enter the name
        search_box = driver.find_element(By.ID, "srch-qry")
        search_box.clear()
        search_box.send_keys(name)
        search_box.send_keys(Keys.RETURN)

        # Wait for results to load
        time.sleep(5)

        # Check if there are no results
        no_results = driver.find_elements(By.ID, "no-results")
        if no_results:
            return None  # No results found

        # Initialize an empty list to store the results
        results = []

        # Extract the main result area
        while True:
            result_elements = driver.find_elements(By.CSS_SELECTOR, "#main_area_div .result-name, .result-metadata")

            for i in range(0, len(result_elements), 3):
                judge_name = result_elements[i].text
                position = result_elements[i + 1].text if (i + 1) < len(result_elements) else ""
                court_name = result_elements[i + 2].text if (i + 2) < len(result_elements) else ""
                results.append([name, judge_name, position, court_name])

            # Check if the "Load More" button exists
            try:
                load_more_button = driver.find_element(By.CLASS_NAME, "load-more-btn")
                load_more_button.click()
                time.sleep(3)  # Wait for more results to load
            except:
                break  # No more results

        return results

    except Exception as e:
        print(f"Error searching for {name}: {e}")
        return None

In [14]:
# Open the CSV file to store the results
with open("judge_search_results.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Search Name", "Judge Name", "Position", "Court Name"])

    # Iterate over the list of names and search each
    for name in judges[:5]:
        results = search_name(name)
        if results:
            writer.writerows(results)
        else:
            writer.writerow([name, "", "", ""])  # No results found

# Close the browser once done
driver.quit()

NameError: name 'csv' is not defined